In [1]:
import os
import sys
import delta
from pyspark.sql.window import Window
import pyspark.sql.functions as psf
import pandas as pd
import pyspark.sql
from pyspark.sql.types import StructType, StructField, DateType, \
    StringType, DoubleType, TimestampType

sys.path.insert(0, os.path.abspath('./source/Producto_DIA_DP'))

from SparkDBUtils import SparkDB
sparkdb = SparkDB()
spark = sparkdb.spark

In [4]:
spark.sql("show schemas").show(truncate=False)

+----------------+
|namespace       |
+----------------+
|analisis_precios|
|default         |
|producto_dia    |
+----------------+



In [5]:
spark.sql("show tables in producto_dia").show(truncate=False)

+------------+---------------+-----------+
|namespace   |tableName      |isTemporary|
+------------+---------------+-----------+
|producto_dia|producto_dim   |false      |
|producto_dia|sequences_cfg  |false      |
|producto_dia|staging_product|false      |
+------------+---------------+-----------+



In [2]:
df = spark.table("producto_dia.staging_product").toPandas()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_producto  0 non-null      object
 1   url_product  0 non-null      object
 2   index        0 non-null      int32 
dtypes: int32(1), object(2)
memory usage: 0.0+ bytes


In [2]:
df = spark.table("producto_dia.producto_dim").toPandas()
df.info()

c:\users\carlos\proyectos\foodecommercescraper\venv\lib\site-packages\pyspark\sql\pandas\conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   product_id  200 non-null    object        
 1   product     190 non-null    object        
 2   brand       190 non-null    object        
 3   categories  200 non-null    object        
 4   units       195 non-null    object        
 5   price       195 non-null    float64       
 6   unit_price  195 non-null    float64       
 7   discount    0 non-null      float64       
 8   date        200 non-null    object        
 9   ts_load     200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 15.8+ KB


In [4]:
df.head(20)

,product_id,product,brand,categories,units,price,unit_price,discount,date,ts_load
0,267616,Champú split hair miracle cabello encrespado,Gliss,"[Perfumería. higiene. salud, Champú]",€/100 ML.),2.99,1.20,NaN,2023-09-12,2023-09-12 06:10:04.182413
1,268029,Colonia sonata dulce,Melody,"[Perfumería. higiene. salud, Colonias]",€/100 ML.),3.99,3.99,NaN,2023-09-12,2023-09-12 06:10:04.604319
2,268553,Verduras con cebada al toque de eneldo,Al,"[Congelados, Verduras. hortalizas y salteados]",€/KILO),1.59,3.98,NaN,2023-09-12,2023-09-12 06:10:04.948069
3,269253,Aditivo desinfectante textil,Lysol,"[Limpieza y hogar, Lejía y otros químicos]",€/LITRO),2.71,3.76,NaN,2023-09-12,2023-09-12 06:10:05.291818
4,269405,Friegasuelos aroma spa,Super,"[Limpieza y hogar, Cristales y suelos]",€/LITRO),1.09,0.73,NaN,2023-09-12,2023-09-12 06:10:05.526196
5,269879,Tortitas de arroz integral con chocolate blanco,Bicentury,"[Galletas. bollos y cereales, Tortitas]",€/KILO),2.49,18.04,NaN,2023-09-12,2023-09-12 06:10:05.869912
6,270472,Exfoliante corporal de coco,Body,"[Perfumería. higiene. salud, Hidratación corpo...",€/100 ML.),4.99,2.50,NaN,2023-09-12,2023-09-12 06:10:06.213667
7,270997,Agua de colonia clásica fresh,S3,"[Perfumería. higiene. salud, Colonias]",€/100 ML.),3.45,0.46,NaN,2023-09-12,2023-09-12 06:10:06.526166
8,271099,Pepito fondant,Donuts,"[Galletas. bollos y cereales, Bollería]",€/KILO),2.99,12.56,NaN,2023-09-12,2023-09-12 06:10:06.901200
9,271330,Leche fresca semidesnatada,Dia,"[Leche. huevos y mantequilla, Leche]",€/LITRO),1.04,1.04,NaN,2023-09-12,2023-09-12 06:10:07.229286


# DELETE producto_dia.staging_product

In [4]:
dt = delta.DeltaTable.forName(spark, "producto_dia.staging_product")

In [5]:
dt.delete(psf.column("index") > "112")

In [8]:
dt.toDF().toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_producto  113 non-null    object
 1   url_product  113 non-null    object
 2   index        113 non-null    int32 
dtypes: int32(1), object(2)
memory usage: 2.3+ KB


# DELETE producto_dia.producto_dim

In [3]:
dt = delta.DeltaTable.forName(spark, "producto_dia.producto_dim")

In [4]:
dt.delete()

In [5]:
dt.toDF().show()

+----------+-------+-----+----------+-----+-----+----------+--------+----+-------+
|product_id|product|brand|categories|units|price|unit_price|discount|date|ts_load|
+----------+-------+-----+----------+-----+-----+----------+--------+----+-------+
+----------+-------+-----+----------+-----+-----+----------+--------+----+-------+

